In [ ]:
# universal_reader_service.py

import os
from pathlib import Path
from typing import Union, List

# Import Agno readers
from agno.knowledge.reader.csv_reader import CSVReader
from agno.knowledge.reader.pdf_reader import PDFReader
from agno.knowledge.reader.docx_reader import DocxReader
from agno.knowledge.reader.pptx_reader import PPTXReader
from agno.knowledge.reader.json_reader import JSONReader
from agno.knowledge.reader.text_reader import TextReader

# Optional future readers:
# from agno.knowledge.reader.html_reader import HTMLReader
# from agno.knowledge.reader.excel_reader import ExcelReader


class UniversalReaderService:
    """Service to read multiple file formats using Agno readers."""

    def __init__(self):
        # Initialize all available readers
        self.readers = {
            ".pdf": PDFReader(),
            ".csv": CSVReader(),
            ".docx": DocxReader(),
            ".pptx": PPTXReader(),
            ".json": JSONReader(),
            ".txt": TextReader(),
        }

    def read(self, file_path: Union[str, Path]) -> List:
        """Read a file using the correct Agno reader."""
        path = Path(file_path)

        if not path.exists():
            raise FileNotFoundError(f"❌ File not found: {path}")

        ext = path.suffix.lower()

        if ext not in self.readers:
            raise ValueError(f"❌ Unsupported file type: {ext}")

        reader = self.readers[ext]

        # Important: pass Path object, not string
        try:
            docs = reader.read(path)
            return docs
        except Exception as e:
            print(f"⚠️ Error reading {path.name} with {reader.__class__.__name__}: {e}")
            return []

    def extract_text(self, docs: List) -> str:
        """Concatenate all document chunks into plain text."""
        content = ""
        for doc in docs:
            # Each doc is typically an Agno Document object with .content
            if hasattr(doc, "content"):
                content += doc.content + "\n"
        return content.strip()


# Example usage
if __name__ == "__main__":
    service = UniversalReaderService()

    # Example 1: PDF
    pdf_docs = service.read("Gurbhej_Data_Science.pdf")
    pdf_text = service.extract_text(pdf_docs)
    print(f"PDF content length: {len(pdf_text)}")

    # # Example 2: CSV
    # csv_docs = service.read("dushu.csv")
    # csv_text = service.extract_text(csv_docs)
    # print(f"CSV content length: {len(csv_text)}")

    # # Example 3: DOCX
    # docx_docs = service.read("Flask Error Handling Architecture Documentation.docx")
    # docx_text = service.extract_text(docx_docs)
    # print(f"DOCX content length: {len(docx_text)}")

    # # Example 4: JSON
    # json_docs = service.read("data.json")
    # json_text = service.extract_text(json_docs)
    # print(f"JSON content length: {len(json_text)}")

    # # Example 5: PPTX (if any)
    # pptx_docs = service.read("slides.pptx")
    # pptx_text = service.extract_text(pptx_docs)
    # print(f"PPTX content length: {len(pptx_text)}")


In [ ]:
# finance_team_agents.py

from textwrap import dedent
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.team import Team


# === 1️⃣ Data Analyst Agent ===
data_analyst_agent = Agent(
    name="Data Analyst Agent",
    role="Analyze structured and unstructured financial data",
    model=OpenAIChat(id="gpt-5-mini",
                     api_key = "OPENAI_API_KEY"),
    instructions=dedent("""\
        You are an expert financial data analyst responsible for examining company and market data. 📊

        Your core responsibilities:
        1. Analyze financial statements (income, balance sheet, cash flow)
        2. Identify revenue trends, cost structures, and profitability ratios
        3. Highlight anomalies or unusual spikes in metrics
        4. Present Key Performance Indicators (KPIs) in a concise summary
        5. Compute YoY and QoQ growth where possible

        Your style guide:
        - Use clean Markdown tables for numbers
        - Highlight trends with arrows (⬆️ ⬇️)
        - Include short comments beside each key metric
        - Use bullet points for insights
        - End with a 'Summary of Financial Health' section\
    """),
    markdown=True,
)


# === 2️⃣ Risk Evaluator Agent ===
risk_evaluator_agent = Agent(
    name="Risk Evaluator Agent",
    role="Evaluate financial and operational risks",
    model=OpenAIChat(id="gpt-5-mini"),
    instructions=dedent("""\
        You are a risk management expert. ⚠️ Your job is to assess company-level and macroeconomic risks.

        Your core responsibilities:
        1. Evaluate liquidity, leverage, and cash flow stability
        2. Assess exposure to market, credit, and operational risks
        3. Identify potential red flags from the data analyst’s findings
        4. Estimate risk severity (Low / Moderate / High)
        5. Recommend mitigation strategies

        Your style guide:
        - Present risk factors in a structured table
        - Use emojis (🟢🟡🔴) for severity levels
        - Keep tone formal and data-driven
        - Conclude with 'Top 3 Risks' and mitigation recommendations\
    """),
    markdown=True,
)


# === 3️⃣ Market Strategist Agent ===
market_strategist_agent = Agent(
    name="Market Strategist Agent",
    role="Develop strategic recommendations and investment insights",
    model=OpenAIChat(id="gpt-5-mini"),
    instructions=dedent("""\
        You are a senior market strategist providing actionable insights for investors. 💡

        Your core responsibilities:
        1. Use insights from both the Data Analyst and Risk Evaluator
        2. Formulate investment or strategic recommendations
        3. Identify sectors or opportunities for growth
        4. Highlight long-term vs short-term outlooks
        5. Provide a concluding 'Strategic Summary' section

        Your style guide:
        - Start with a concise 'Market Overview'
        - Provide 2–3 actionable investment insights
        - Use clear headers for each strategic area
        - End with 'Final Recommendations' and bullet points
        - Maintain a confident, advisor-like tone\
    """),
    markdown=True,
)


# === 4️⃣ Finance Team (collaboration) ===
finance_team = Team(
    members=[data_analyst_agent, risk_evaluator_agent, market_strategist_agent],
    model=OpenAIChat(id="gpt-5-mini"),
    name="Finance Analysis Team",
    description="A collaborative multi-agent system for financial analysis, risk evaluation, and market strategy.",
    instructions=dedent("""\
        You are the lead editor and coordinator of a financial intelligence desk! 🧠

        Your responsibilities:
        1. Coordinate between the Data Analyst, Risk Evaluator, and Market Strategist
        2. Combine their findings into a single cohesive report
        3. Maintain consistent tone, structure, and formatting
        4. Validate all numerical data and logical reasoning
        5. Provide a balanced view of financial health and outlook

        Your style guide:
        - Begin with an 'Executive Summary'
        - Use clear section headers: Financial Analysis | Risk Assessment | Market Strategy
        - Present key data and insights in tables and bullet points
        - Add a 'Key Takeaways' section at the end
        - Finish with a short 'Conclusion' paragraph signed off as 'Finance Analysis Team' with today's date
        - Maintain formal, analytical language\
    """),
    add_datetime_to_context=True,
    markdown=True,
    show_members_responses=True,  # change to False to suppress intermediate outputs
)


# === Example Usage ===
if __name__ == "__main__":
    finance_team.print_response(
        input="Analyze the Q3 2024 performance of Tesla (TSLA), assess its key risks, and provide investment recommendations.",
        stream=True,
    )

    # finance_team.print_response(
    #     input="Review Apple Inc. (AAPL) FY2024 financials and identify potential risks and long-term strategic opportunities.",
    #     stream=True,
    # )

    # finance_team.print_response(
    #     input="Provide a comprehensive report on the semiconductor industry (NVIDIA, AMD, TSMC) focusing on financial strength, risk exposure, and strategic positioning.",
    #     stream=True,
    # )
